In [1]:
import pandas as pd

# 데이터분석 스케치

## [Horse colic](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+)

+ team 
  + 권순호, 윤지원, 김원정
  

+ index
	* EDA(Exploratory data analysis)
	* Feature selection
	* Modeling
	* Model selection

## EDA
+ 변수 탐색 및 분류
  + 변수 이름과 설명
  + 상관관계가 없는 변수는 어떤것이 있나?
  + 의미없는값, 전처리가 필요한 값들을 선별및 가공하자

In [30]:
df = pd.read_csv('./horse.csv')

In [33]:
df.tail()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
294,yes,adult,533886,NaN,120.0,70.0,cold,NaN,pale_cyanotic,more_3_sec,...,55.0,65.0,NaN,NaN,euthanized,no,3205,0,0,no
295,no,adult,527702,37.2,72.0,24.0,cool,increased,pale_cyanotic,more_3_sec,...,44.0,NaN,serosanguious,3.3,euthanized,yes,2208,0,0,yes
296,yes,adult,529386,37.5,72.0,30.0,cold,reduced,pale_cyanotic,less_3_sec,...,60.0,6.8,NaN,NaN,died,yes,3205,0,0,no
297,yes,adult,530612,36.5,100.0,24.0,cool,reduced,pale_pink,less_3_sec,...,50.0,6.0,serosanguious,3.4,lived,yes,2208,0,0,yes
298,yes,adult,534618,37.2,40.0,20.0,NaN,NaN,NaN,NaN,...,36.0,62.0,clear,1.0,euthanized,no,6112,0,0,no


+ 변수 이름과 설명:

  1:  surgery?(수술 경험)
       1 = Yes, it had surgery
       2 = It was treated without surgery

  2:  Age(나이)
       1 = 성인 말
       2 = 어린 말 (< 6개월)

  3:  Hospital Number(병원 번호)
       - 숫자 ID
       - 말에 지정된 case number
            (만약 말을 1회 이상 치료한다면 특별하지 않을 수 있다.)

  4:  rectal temperature(직장 온도)
       - linear
       - in degrees celsius.
       - 감염으로 인해 고온이 발생할 수 있습니다.
       - 동물이 late shock을 받으면 온도가 낮아질 수 있다.
       - 정상 온도는 37.8
       - 이 매개 변수는 일반적으로 문제가 진행됨에 따라 변경됩니다. (예를 들어, 말이 충격에 빠질 때 정상 범위를 통과하면서 병변 때문에 상승할 수 있습니다.)
  5:  pulse(맥박)
       - linear
       - 분당 심장 박동 수
       - is a reflection of the heart condition: 30-40은 성인에게 정상입니다.
       - 경주용 말이지만 일반 말보다 심장박동수가 낮다(분당 20-25정도)
       - 통증이있는 병변이 있거나 순환계 충격을 겪은 동물은 심박수가 상승 할 수 있습니다

  6:  respiratory rate(호흡 속도)
       - linear
       - normal rate is 8 to 10
       - 심한 변동 때문에 유용성이 의심스럽습니다.

  7:  temperature of extremities(팔 다리의 온도)
       - 말초 혈액 순환의 주관적 표시
       - possible values:
         1 = Normal
         2 = Warm
         3 = Cool
         4 = Cold
       - cool와 cold는 쇼크의 가능성을 보여줍니다.
       - 고열 사지는 직장 온도 상승과 관련이 있습니다.

  8:  peripheral pulse(말초 맥박)
       - subjective
       - possible values are:
         1 = normal
         2 = increased
         3 = reduced
         4 = absent
       - normal or increased p.p.는 적절한 순환을 나타내며 reduced or absent는 나쁜 perfusion을 나타냅니다.

  9:  mucous membranes(점막)
       - 주관적인 색의 측정
       - possible values are:
         1 = normal pink
         2 = bright pink
         3 = pale pink
         4 = pale cyanotic
         5 = bright red / injected
         6 = dark cyanotic
       - 1, 2는 정상 또는 약간 증가 된 혈액 순환을 나타냅니다
       - 3 는 early shock 발생일수도 있습니다.
       - 4,6 은 심각한 순환 기관의 손상을 나타낸다.
       - 5는 패혈증을 더 많이 나타낸다.

 10: capillary refill time(모세관 주입 시간)
       - 임상 적 판단. 리필 시간이 길수록 혈액 순환이 원활하지 않음.
       - possible values
         1 = < 3 seconds
         2 = >= 3 seconds

 11: pain 
       - 말의 통증 수준에 대한 주관적인 판단
       - possible values:
         1 = alert, no pain
         2 = depressed
         3 = intermittent mild pain
         4 = intermittent severe pain
         5 = continuous severe pain
       - **should NOT be treated as a ordered or discrete variable!**
       - 일반적으로 통증이 심할수록 수술이 필요할 가능성이 높습니다.
       - 통증을 미리 치료하면 통증 수준을 어느 정도 가릴 수 있습니다.

 12: peristalsis(연동운동)                        
       - 말의 장내 활동의 표시. 내장이 점점 팽창하거나 말이 더 독성이 생기면서 활동은 감소한다.
       - possible values:
         1 = hypermotile
         2 = normal
         3 = hypomotile
         4 = absent

 13: abdominal distension(복부 팽창)
       - ** An IMPORTANT parameter.**
       - possible values
         1 = none
         2 = slight
         3 = moderate
         4 = severe
       - 복부 팽만감을 가진 동물은 고통스럽고 내장 운동성을 감소시킬 수 있습니다.
       - 복부 팽창이 심한 말은 단지 압력을 완화시키기 위해 수술이 필요할 것이다.

 14: nasogastric tube(비위 관)
       - 이것은 튜브에서 나오는 가스를 의미합니다.
       - possible values:
         1 = none
         2 = slight
         3 = significant
       - 위에 있는 큰 가스 캡은 말을 불편하게 할 수 있다.

 15: nasogastric reflux(비위 역류)
       - possible values
         1 = none
         2 = > 1 liter
         3 = < 1 liter
       - 역류 량이 많을수록 장의 나머지 부분으로부터의 유체 통로에 심각한 장애물이있을 확률이 높습니다

 16: nasogastric reflux PH(비위 관성 환류 PH)
       - linear
       - scale is from 0 to 14 with 7 being neutral
       - 정상 값은 3에서 4 범위에 있습니다.

 17: rectal examination - feces(직장 검사 - 대변)
       - possible values
         1 = normal
         2 = increased
         3 = decreased
         4 = absent
       - 배설물이 없다는 것은 아마도 장애를 나타냅니다.

 18: abdomen(복부)
       - possible values
         1 = normal
         2 = other
         3 = firm feces in the large intestine
         4 = distended small intestine
         5 = distended large intestine
       - 3은 아마도 기계적인 충격에 의해 야기된 방해물일 것이다, 그리고 보통 의학적으로 치료된다.
       - 4 및 5는 수술 병변을 나타낸다


 19: packed cell volume
          - linear
          - the # of red cells by volume in the blood
          - normal range is 30 to 50. The level rises as the circulation
            becomes compromised or as the animal becomes dehydrated.

 20: total protein
          - linear
          - normal values lie in the 6-7.5 (gms/dL) range
          - the higher the value the greater the dehydration

 21: abdominocentesis appearance
          - a needle is put in the horse's abdomen and fluid is obtained from
            the abdominal cavity
          - possible values:
               1 = clear
               2 = cloudy
               3 = serosanguinous
          - normal fluid is clear while cloudy or serosanguinous indicates
            a compromised gut

 22: abdomcentesis total protein
          - linear
          - the higher the level of protein the more likely it is to have a
            compromised gut. Values are in gms/dL

 23: outcome
          - what eventually happened to the horse?
          - possible values:
               1 = lived
               2 = died
               3 = was euthanized

 24: surgical lesion?
          - retrospectively, was the problem (lesion) surgical?
          - all cases are either operated upon or autopsied so that
            this value and the lesion type are always known
          - possible values:
               1 = Yes
               2 = No

 25, 26, 27: type of lesion
          - first number is site of lesion
               1 = gastric
               2 = sm intestine
               3 = lg colon
               4 = lg colon and cecum
               5 = cecum
               6 = transverse colon
               7 = retum/descending colon
               8 = uterus
               9 = bladder
               11 = all intestinal sites
               00 = none
          - second number is type
               1 = simple
               2 = strangulation
               3 = inflammation
               4 = other
          - third number is subtype
               1 = mechanical
               2 = paralytic
               0 = n/a
          - fourth number is specific code
               1 = obturation
               2 = intrinsic
               3 = extrinsic
               4 = adynamic
               5 = volvulus/torsion
               6 = intussuption
               7 = thromboembolic
               8 = hernia
               9 = lipoma/slenic incarceration
               10 = displacement
               0 = n/a
 28: cp_data
          - is pathology data present for this case?
               1 = Yes
               2 = No
          - this variable is of no significance since pathology data
            is not included or collected for these cases

8. Missing values: 30% of the values are missing

## Feature selection

## Modeling

## Model selection